# Project 3, Part 2: Deep Q Learning with Gym and Tensorflow

## Q, who?

Deep Q learning is going to be our first 'real' reinforcement learning algorithm and the most difficult machine learning concept we've tried to cover so far. I've actually yet to find an intutive explanation of how it works, so we're going to have to get a little more theoretical than usual. I'll also note that this isn't something that has libraries just laying around. We're definitely going to use tools that you've already seen, but this is going to be a little more 'manual' than our other notebooks.

Let's start building our foundations:

### Markov Descision Processes

A Markov desicion process is a diagram that represents every possible action in an environment. It's a graph where the nodes represent a state, the connections are potential actions, each action has a probability of entering certain other states, and the transition to some states yield rewards. 

![Markov Descision Processes](https://cdn-images-1.medium.com/max/1200/1*QuBOz2yQ5Fy6YnZyvSPXzw.png)

Ok, let's examine some hallmarks of this Markov Process.

- Let's start with our first state, S0. From S0 we can make 2 actions: a0 or a1. The a0 action has a 50/50 chance of either yielding S0 or S2 and the a1 action will lead to S2 100% of the time. 
- S2, a1 yields a 30% chance of transitioning to S0 and -1 reward.
- S1, a0 has a 70% chance of going to S0 with a +5 reward.

From S0, how would you make it to the +5? The best path seems to be S0, a1 to S2, a1 to S1, a0. But there's obviously no guaranteed chance that you make it to the +5 on your first try. The S2, a1 action has equal likelihood of losing a point as it does transitioning to S1.

- What if your agent hit the -1 at S2, a1? Would it never try that action again?
- What if this was a game like cartpole where you got a point just for not failing? Would it ever even 'go looking' for the +5?

### Solving all our problems

We've a stack of equations to get through before we can implement Deep Q learning, but I'm going to do my best to abstract as much as possible. 

1. The Bellman Optimality Equation
    1. This equation states that the optimal state is the optimal state plus the expected optimal values of all possible next states after the inital optimal action.
    2. This equation assumes we know all of the states, probabilites and rewards.
    3. Notice how I said 'optimal state' and not 'optimal action'.
    
2. Value Iteration Algorithm
    1. An algorithm that is used to develop your Bellman Optimality Equation, when you don't already know all of the states, probabilites, and rewards.
    2. Notice that this solves 1.2 but not 1.3.
    
3. **Q-Value** Iteration Algorithm
    1. A Q-Value is a representation of the optimal state-action value. Q-Value is notated as Q(s,a). It's a sum of all **discounted** furture rewards that an agent will gain after is reaches state s and chooses action a.
    2. **Discounted** means that we reduce the importance of rewards that are further in the future. This will be a tuneable hyperparameter later on.
        - **hyperparameter**: I'm not sure if we've defined this yet, but a 'hyperparameter' is a machine learning paramter than can be tuned by you, the human. The weights between the neurons in a NN would be an example of just a 'parameter'.
        - Side note: This discount is the γ (gamma) charater you'll see if you look the equation up. It's important enough that this is worth knowing if you go read other guides (ha! like you'd ever need that....).
    3. The Q-Value Iteration Algorithm builds out a table of the optimal Q value values for the entire Markov Descision Process.
    4. Notice that this solves the problem in 1.3. Q Values are concerned with optimal actions, not optimal states.
    
4. **Q-Learning** Algorithm
    1. If the Q-Value Iteration Algorithm is the Bellman Optimality Equation, then the Q-Learning Algorithm is the Value Iteration Algorithm. It's basically the way we go about building out our Q Values when we don't already know all of the tranistion probabilities and rewards. 
    
With this stack of concepts, building out a policy for our environment is as easy as saying "Give me the highest Q value" at each state that we encounter. Some pseudo code might look like:

```python
action = max(Q(s, a))
obs, reward, done, info = env.step(action)
```

### Approximate Q-Learning

There's a problem with "Solving all our problems" point 3.3. Notice how the Q-Value iteration algorithm creates a table of Q-Values? Well, unfourtunately this isn't feasible for environments large enough to be interesting. 

Also, there's some hint in that this is a machine learning notebook. All of the Q Learning concepts we've discussed so far are deterministic. You could implement the equations we've been talking about with normal coding technique: Build a Q Value table using the Q-Learning Algorithm then build an agent that just looks up the highest Q Value for a given state. The `max()` part of my pseudo code earlier was literal.

##### Enter, Approximate Q-Learning

As I just hinted, most environments of any interest are too complicated for good ol' fashioned Q-Learning. What we actually want is approximate Q-Learning.

Rather than knowing the actual Q Value for every possible state, action pair, we can just do our best to approximate the optimal Q Value. This can accutally be accomplished with multiple deep learning techniques, but we'll be using a Deep Q (Neural) Network, or DQN.

## Deep Q-Learning

I'm sorry to do this to you, but this really is easier with the equation:

```
y(s,a) = r + γ * max(Q(s',a'))
```

Alright, here's the parts of the equation:

- y(s,a): The target Q-Value. Think of this as the label in our other machine learning notebooks.
    - s: Current state.
    - a: Current action
- r: The reward we were given for our action
- γ: Future discount hyperparameter
- max(Q(s',a')): Estimated Q-Value for our next state and action.
    - s': Next state
    - a': Next action

The order of operations here is:

1. Observe that we're in sate (s).
2. Execute action (a).
3. Learn reward (r).
4. Learn our new state (s').
5. Run s' through our DQN to get our new Q-Value action (a').
6. Multiply this future Q-Value, max(Q(s',a')), by our future discount (γ).
7. Add our reward (r).

We now have a approximate Q-Value for our previous state (y(s,a)) and can use that as a label for training our neural network.

### Replay

Researchers have found that we don't just want to take every new action and run it through the neural network for training. What we need is memory and replay.

There's 2 main issues we're addressing with memory and replay:

1. Our DQN will 'forget' what it previously learned. Let's say we're training on a video game with levels. Our DQN trains until it passes the first level and is now on the second level. It's terrible at the second level, but slowly learns to beat it. That same DQN probably couldn't go back and beat the first level. It needed to retrain all of the weights between its neurons in order to learn the 2nd level.
2. Actions have consequences. Your first action effects your second state, your second action effects your third state, etc. We don't want to learn and favor this correlated data, we want to know the best approximate Q value no matter the current state.
    - Let's say we're playing a poorly designed wack-a-mole. There's 2 holes: Left and right. Wacking a mole results in an 85% probability that the next mole will be on that same side. Your DQN will slowly learn to favor one side, for no other reason that that's what it chose first.
    
We get around this with **replay memory**. We'll be storing our experiecnes in a replay buffer, then sampling those experiences at random during training. This mixing of experiences reduces our issues with forgetting and with experiential correlation.

### Dueling Networks

Ok, not really dueling...but we also need two networks. Part of what makes the Q-Learning equation hard to wrap your mind around is that it's using the same network to predict values that it trains on. It's pretty easy to end up chasing your own tail.

What we're going to do is have a model and a target_model. The model is going to be used to play and the target model will be used to estimate the Q-Value. The target_model will only be updated when the game is finised.

We get a more stable, consistent training this way.

### Implementing our solutions

All of the above sections give us theoretical underpinnings, but they don't really tell us how to implement DQNs.

First of all, how do we actually explore the environment?

We could use something like we used in our last notebook: A completely random policy. But there's better ways. Why not deisgn something that purposely explores? 

We want an **ε-greedy** policy.  
   - That character is called epsilon. It's another one that's used everywhere so you'll want to recognize it.

In this case, the ε represents the probability that your exploration policy will act randomly. And there's a 1-ε probability that your policy will act greedily (on the best Q Value)

So an ε-greedy Q-Learning Algorithm is one that slowly hones in on and explores the interesting, rewarding parts of the environment while occassionally still jumping into a fire, just to make sure it doesn't want to go that way. 

Note that we also typically slowly decrease ε over time. This is called out **decay rate**. A brand new agent really needs to make sure that the fire isn't good, but an old, battle hardened agent has been burned enough times to know better.

## Code time!

You'll find extensive comments in the code below.


In [9]:
from collections import deque
import gym
import numpy as np
import random
from tensorflow import keras

# Building and training our DQN is much more complicated that our other models so far
# Hence, the class.
class DQNAgent:

    def __init__(self):
        # deque keeps our memory size under control.
        # When item 2001 is added, item 1 is deleted.
        self.memory = deque(maxlen=2000)
        # How much we are discounting furture rewards
        self.gamma = 0.99
        # Likelihood of acting randomly 
        self.epsilon = 1.0
        # The rate at which epsilon will go down.
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.0001
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
        

    def _build_model(self):
        model = keras.Sequential()
        model.add(keras.layers.Dense(32, activation='relu', input_dim=4))
        model.add(keras.layers.Dense(16, activation='relu'))
        model.add(keras.layers.Dense(8, activation='relu'))
        model.add(keras.layers.Dense(4, activation='relu'))
        # Linear, because we want bigger to equal better
        model.add(keras.layers.Dense(2, activation='linear'))
        model.compile(optimizer=keras.optimizers.Adam(lr=self.learning_rate), loss='mse')
        return model
    
    
    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

        
    # Using 'state' rather than 'observation' to jive with the Q-lingo
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
        
    def act(self, state):
        # If random number between 0 and 1 <= our likelihood of random action
        if np.random.rand() <= self.epsilon:
            # Act randomly
            return np.random.randint(2)
        # If not acting randomly, actually try to predict our action
        action_values = self.model.predict(state)
        return np.argmax(action_values[0])
    
    
    def replay(self, batch_size):
        mini_batch = random.sample(self.memory, batch_size)
        
        # We're going to make some some taget and value arrays. Initializing values
        update_state = np.zeros((batch_size, 4))
        update_next_state = np.zeros((batch_size, 4))
        action, reward, done = [], [], []
        # Building taget and value arrays.
        for i in range(batch_size):
            update_state[i] = mini_batch[i][0]
            action.append(mini_batch[i][1])
            reward.append(mini_batch[i][2])
            update_next_state[i] = mini_batch[i][3]
            done.append(mini_batch[i][4])

        # Getting our predicted actions for state and next state
        target = self.model.predict(update_state)
        target_next = self.model.predict(update_next_state)
        target_val = self.target_model.predict(update_next_state)

        for i in range(batch_size):
            # If done, there is no relevant next value
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                a = np.argmax(target_next[i])
                # Notice that this is the q-value equation from earlier
                target[i][action[i]] = reward[i] + self.gamma * (
                    target_val[i][a])

        # Train our model with our q-value
        self.model.fit(update_state, target, batch_size=batch_size,
                       epochs=1, verbose=0)
        
        # Epsilon Decay
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def save(self, name):
        self.model.save(name)
            

"""
Now let's train our DQN
"""

# Notice that I'm slightly changing how we initialize the env.
# I'm doing this so that we can increase the max score
env = gym.envs.make('CartPole-v1')
env._max_episode_steps = 10000
agent = DQNAgent()

for episode in range(500):
    done = False
    state = env.reset()
    state = np.reshape(state, [1, 4])
    
    reward_total = 0

    while not done:
        action = agent.act(state)
        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state, [1, 4])
        # Cartpole doesn't have negative rewards, so we're making our own.
        # We want to strongly disincentivize the actions leading up to a failure.
        reward = reward if not done or reward_total == 9999 else -100
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        reward_total += reward
        if done:
            agent.update_target_model()
            if episode % 10 == 0:
                print(f'episode:{episode}, reward: {reward_total}, epsilon: {round(agent.epsilon, 5)}')
            break
        if len(agent.memory) > 64:
            agent.replay(64)
            
agent.save('cartpole-dqn1.h5')
        

episode:0, reward: -73.0, epsilon: 1.0
episode:10, reward: -58.0, epsilon: 0.82771
episode:20, reward: -85.0, epsilon: 0.63051
episode:30, reward: -93.0, epsilon: 0.50392
episode:40, reward: -65.0, epsilon: 0.39319
episode:50, reward: 51.0, epsilon: 0.17713
episode:60, reward: 118.0, epsilon: 0.01457
episode:70, reward: 107.0, epsilon: 0.00168
episode:80, reward: 73.0, epsilon: 0.00029
episode:90, reward: 66.0, epsilon: 0.0001
episode:100, reward: 51.0, epsilon: 0.0001
episode:110, reward: 56.0, epsilon: 0.0001
episode:120, reward: 63.0, epsilon: 0.0001
episode:130, reward: 87.0, epsilon: 0.0001
episode:140, reward: 93.0, epsilon: 0.0001
episode:150, reward: 90.0, epsilon: 0.0001
episode:160, reward: 111.0, epsilon: 0.0001
episode:170, reward: 237.0, epsilon: 0.0001
episode:180, reward: 320.0, epsilon: 0.0001
episode:190, reward: 421.0, epsilon: 0.0001
episode:200, reward: 183.0, epsilon: 0.0001
episode:210, reward: 135.0, epsilon: 0.0001
episode:220, reward: 196.0, epsilon: 0.0001
epi

## DQN Training Results

Well, it's definilty odd. You can see a slow increase in the score over time, but it doesn't always stay consistently better. 

The highest recorded result was 6453 and we ended on -32 (remember that we give a large negative reward for failure).

I'm honestly not sure why that is. There is some tiny amount of randomness, but epsilon was at its minimum by episode 90. Maybe DQNs just need alot more training time to get good. Remember that this is the first truly exploratory policy we've had. We didn't just feed in good data.

Let's see how the DQN does with no randomness and compare to our other policies:

In [12]:
import gym
import numpy as np
from tensorflow import keras

env = gym.make('CartPole-v1')
policy = keras.models.load_model('cartpole-dqn1.h5')
totals = []

for episode in range(10000):
    episode_reward_total = 0
    state = env.reset()
    for step in range(500):
        action = np.argmax(policy.predict(state.reshape(1, 4))[0])
        state, reward, done, info = env.step(action)
        episode_reward_total += reward
        if done:
            break

    totals.append(episode_reward_total)

print(f'Max: {max(totals)}')
print(f'Min: {min(totals)}')
print(f'Average: {round(sum(totals) / len(totals), 0)}')

Max: 500.0
Min: 197.0
Average: 279.0


### Disappointingly low. 

It's better than our manual polices, but worse than our first NN:

|Policy |Min  |Max  |Average |
|-------|-----|-----|--------|
|Dumb 1 |24   |72   |42      |
|Dumb 2*|8    |500  |50      |
|Neural |222  |500  |490     |
|DQN1   |197  |500  |279     |

I'm not 100% sure what our issue is, but I think our first step should be way more training.

## Getting beefy with it

I want to see how a really large training will go. We're going to:
1. Train with much, much large parameters
2. See how we stack up with our standard test
3. Run the standard test but with greatly increased potential max score.

We aren't actually going to train this DQN in this notebook. I want to see the env render along the way, so I'm including a file named 'dqn.py' in this folder. This uses pretty much the same code as DQN1 with a few additions:

1. Training episodes increased from 500 to 10000
2. The training will stop and save if the mean of the most recent games is close to max.
3. Renders the game on your screen as the DQN is playing it. (remember that you can't render in jupyter)
4. Will save another model to your local folder if training loop finishes.

I highly encourage you to run the code. It doesn't take long to train and it is really neat to acutally get a visual of what the network is doing.

### DQN2 test:

I'm going to be running dqn.py outside of the notebook then coming back here to test the resulting model:

In [1]:
#Testing
import gym
import numpy as np
from tensorflow import keras

env = gym.make('CartPole-v1')
policy = keras.models.load_model('cartpole-dqn2.h5')
totals = []

for episode in range(10000):
    episode_reward_total = 0
    state = env.reset()
    for step in range(500):
        action = np.argmax(policy.predict(state.reshape(1, 4))[0])
        state, reward, done, info = env.step(action)
        episode_reward_total += reward
        if done:
            break

    totals.append(episode_reward_total)

print(f'Max: {max(totals)}')
print(f'Min: {min(totals)}')
print(f'Average: {round(sum(totals) / len(totals), 0)}')

C:\Users\seanm\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Max: 500.0
Min: 500.0
Average: 500.0


### Results

|Policy |Min  |Max  |Average |
|-------|-----|-----|--------|
|Dumb 1 |24   |72   |42      |
|Dumb 2 |8    |500  |50      |
|Neural |222  |500  |490     |
|DQN1   |197  |500  |279     |
|DQN2   |500  |500  |500     |

Well, we did it! We mastered cart-pole using a Deep-Q Network!

## Tweaks

Below is all of the tweaking I ended up making on DQN2. Interestingly, the rendering allowed for much more purposeful tweaking of hyperparameters than ususal.

### Batch Size

I increased the batch training size from 64 to 512. I only knew that I needed to change this because of the render. 

My cart got to the point where it wasn't getting negative scores, but wasn't getting much more than 20 points. It was stuck there for hundreds of iterations. The cart was just going left until it went off screen evey time. 

This was due to the state correlation issue I pointed out earlier in the notebook. My agent learned that it could get points by just going left every time. And sure, there might be a -100 waiting at the end of that decision making process, but my neural net had already learned that left resulted in a non-negative score.

By increasing my batch size, the NN was able to learn on a greater varitey of experiences and was therefore less likely to get stuck in a single strategy.

### Epsilon

I increased the minimum epsilon (the likelihood of random action) from 0.0001 to 0.01.

My agent started getting some really high scores in the 4000-7000 range. But each time it would immediately follow with 5-10 scores that were anywhere from -90-20 (that's not a typo. That's negative 90).

Again, watching the cart helped me figure this out. The cart was really good at keeping the pole balanced once it was totally upright. It didn't even look like the render was moving at all sometimes. What it wasn't good at was recovering from steeper pole angles. 

I theorized that inserting more randomness would force the model to learn to recover its balance and would therefore give me more consistent results - and that's exactly what happened.

### Max Score

I originally increaed the max score for the game to 5000000, but worked my way down to 1000. My thinking was that crazy high scores would make 500 seem easy. 5 million was an insane number to pick anyway, but it was also causing memory issues. A few games lasted into the 100K range, which resulted in the neural net effectively being totally retraind. Much like the epsilon problem, my DQN kept forgetting how to start the game.

I tried lowering to 10K. This was somewhat effective. But after 3 consecutive 10K games my DQN had a streak of 30 negative score games. We were back to forgetting....

1K seemed to be the sweet spot. It's enough that 500 should be trivial but not so much that a single run or two will wipe the DQN's memory.

### Other memory aides

- I also ended up lowering the 'stop and save' threshold down from 10 to 3. 10 consecutive perfect games is excessive and clearly runs the risk of wiping memory.

- After more consideration, even the 10->3 win threshold change didn't seem like enough. I increased the memory size from 2K to 10K. This resulted in a longer peroid of low numbers at the start of training. The replay memory takes longer to forget bad data that the totally random agent generated, but it also lets the agent continue to train on some 'wobbly' data at the end of training. When the agent has 3 consecutive 1K runs 70% of it's training memory is still comprised of the imperfect, but still valuable runs that led to those 3 good runs. 

## The MEGA TEST

Just for fun, let's see how far we can take this. I'm running our standard test, increasing the max score, and decreasing the number of episodes to 10. 

In [6]:
import gym
import numpy as np
from tensorflow import keras

env = gym.envs.make('CartPole-v1')
env._max_episode_steps = 500000
policy = keras.models.load_model('cartpole-dqn2.h5')
totals = []

for episode in range(10):
    episode_reward_total = 0
    state = env.reset()
    for step in range(500000):
        action = np.argmax(policy.predict(state.reshape(1, 4))[0])
        state, reward, done, info = env.step(action)
        episode_reward_total += reward
        if done:
            print(f'episode: {episode}, score: {episode_reward_total}')
            break

    totals.append(episode_reward_total)

print(f'Max: {max(totals)}')
print(f'Min: {min(totals)}')
print(f'Average: {round(sum(totals) / len(totals), 0)}')

episode: 0, score: 500000.0
episode: 1, score: 500000.0
episode: 2, score: 500000.0
episode: 3, score: 500000.0
episode: 4, score: 500000.0
episode: 5, score: 500000.0
episode: 6, score: 500000.0
episode: 7, score: 500000.0
episode: 8, score: 500000.0
episode: 9, score: 500000.0
Max: 500000.0
Min: 500000.0
Average: 500000.0


## Well, alright then

The new DQN had 10 perfect 500,000 runs. We don't have much, if any, room for imporvement. 

In our next notebook we're going shift focus from theory and algorithms to more practical machine learning matters. We're going to explore data pre-processing and using pre-trained model to augment you own. 